In [ ]:
!pip uninstall -y numpy scipy gensim
!pip install numpy==1.24.4 scipy==1.11.4 gensim==4.3.2 --force-reinstall --no-cache-dir

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
Found existing installation: gensim 4.3.2
Uninstalling gensim-4.3.2:
  Successfully uninstalled gensim-4.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 186.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 269.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 176.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 211.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 315.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: smart-open
    Fou

# Mo dau PATH


In [ ]:
# load data

import pandas as pd
data_gome = pd.read_csv('/content/drive/MyDrive/data-raw/processing/data_new.csv')
data_clinc = pd.read_csv('/content/drive/MyDrive/data-raw/processing/data_yd_clinc150.csv')
data_daily = pd.read_csv('/content/drive/MyDrive/data-raw/processing/data_daily_dialog.csv')
data_multi = pd.read_csv('/content/drive/MyDrive/data-raw/processing/data_MultiWoZ.csv')

Tổng hop thu tu hop ly theo nhan thuc cua bộ
não

Giai đoạn

1. Cảm nhận
(Perception) ->           ***  GoEmotions***

2. Hiểu ý định (Intent
Recognition) ->            *** CLINC150***

3. Hiểu ngữ cảnh
(Context Processing) ->    ***MultiWoz, DailyDialog, Persona-Chat***

4. Sinh phản hồi
(Response Generation) ->   ***Tập hội thoại đầy đủ***



In [ ]:
!pip install rank-bm25

# Làm sạch data

In [ ]:
data_gome['clean_text'].isnull().sum()
data_clinc.isnull().sum()
data_daily['text'].isnull().sum()
data_daily.dropna(axis = 0 , inplace= True)
data_multi.isnull().sum()
data_gome = data_gome.dropna(axis=0)

In [ ]:
data_list = data_gome['clean_text'].to_list() + data_clinc['text'].to_list() + data_daily['text'].to_list() +\
            data_multi['text'].to_list()

In [ ]:
len(data_list)

503421

In [ ]:
max([len(x) for x in data_list])

703

# Vocab-tokenizer

In [ ]:
# build vocab -  copus
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import pandas as pd
import numpy as np


tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
train = BpeTrainer(
    special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]" , "[END]" , "[SOS]"]
)
tokenizer.train_from_iterator(data_list, train )

In [ ]:
import pickle

In [ ]:
data_mul_bot = data_multi[data_multi['speaker'] == 0]
data_mul_user = data_multi[data_multi['speaker'] == 1]

emotion_columns = [col for col in data_gome.columns if col not in ['clean_text', "text"]]
data_gome["cls"]  = data_gome[emotion_columns].apply(lambda row: [col for col in emotion_columns if row[col] == 1 ]  , axis=1)
data_gome["cls"] = data_gome["cls"].apply(lambda x: x if x else ["normal"])
data_gome['cls'] = data_gome['cls'].apply(lambda x: ' '.join(x))
data_gome_values = data_gome[['clean_text','cls']].values
data_copus_clinc = data_clinc.values



copus_clinc = [ tokenizer.encode(str(x)).tokens + [e] for x , e in data_copus_clinc ]
copus_go_clinc = {idx : copus_clinc[idx][-1] for idx in range(len(copus_clinc))}
data_Wv = [[x] for x in data_clinc['cls'].unique()]

copus_gomoe = [tokenizer.encode(str(x)).tokens + [e] for x , e in data_gome_values]
copus_go = {idx : copus_gomoe[idx][-1] for idx in range(len(copus_gomoe))}
cls_gomoe = [[x]for x in data_gome["cls"].unique()]

In [ ]:
data_mul_bot.shape

(56776, 2)

#  Lưu model

In [ ]:
set([x[0] == 'amusement annoyance' for x in cls_gomoe])

{False, True}

In [ ]:
bm = BM25Okapi(copus_clinc)
with open('/content/drive/MyDrive/data-raw/processing/BM25O_Retrival_clinc.pkl' , 'wb') as f:
  pickle.dump(bm, f)
bm1 = BM25Okapi(copus_gomoe)
with open('/content/drive/MyDrive/data-raw/processing/BM250_retrival_Gomoe.pkl' , 'wb') as f:
  pickle.dump(bm1, f)



In [ ]:
import json
with open('/content/drive/MyDrive/data-raw/processing/copus_gomoe.json', 'w') as f:
    json.dump(copus_go, f)
with open('/content/drive/MyDrive/data-raw/processing/copus_clinc.json', 'w') as f:
    json.dump(copus_go_clinc, f)


In [ ]:
from gensim.models import Word2Vec,  KeyedVectors
e_label = Word2Vec(cls_gomoe, vector_size=32, window=1, min_count=1, workers=4)
e_label.save('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_gomoe.model')

In [ ]:
from gensim.models import Word2Vec,  KeyedVectors
e_label = Word2Vec(cls_gomoe, vector_size=32, window=1, min_count=1, workers=4)
e_label.save('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_gomoe.model')


e_label_clin  = Word2Vec(data_Wv, vector_size=32, window=1, min_count=1, workers=4)
e_label_clin.save('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_clinc.model' )


In [ ]:
word2vec_clinc = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_clinc.model')
word2vec_gomoe = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_gomoe.model')





In [ ]:
word2vec_gomoe.wv['amusement annoyance']

In [ ]:
max([len(x) for x in data_mul_bot['text']])

# DATA

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# @title
max_token = 200
import numpy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import torch
from rank_bm25 import BM25Okapi
from gensim.models import Word2Vec
import json
# copus_clinc = [ tokenizer.encode(str(x)).tokens + [e] for x , e in data_copus_clinc ]
# data_Wv = [[x] for x in data_clinc['cls'].unique()]


data_mul_bot = data_multi[data_multi['speaker'] == 0]
data_mul_user = data_multi[data_multi['speaker'] == 1]

emotion_columns = [col for col in data_gome.columns if col not in ['clean_text', "text"]]
data_gome["cls"]  = data_gome[emotion_columns].apply(lambda row: [col for col in emotion_columns if row[col] == 1 ]  , axis=1)
data_gome["cls"] = data_gome["cls"].apply(lambda x: x if x else ["normal"])
data_gome['cls'] = data_gome['cls'].apply(lambda x: ' '.join(x))
data_gome_values = data_gome[['clean_text','cls']].values
data_copus_clinc = data_clinc.values



copus_clinc = [ tokenizer.encode(str(x)).tokens + [e] for x , e in data_copus_clinc ]
copus_go_clinc = {idx : copus_clinc[idx][-1] for idx in range(len(copus_clinc))}
data_Wv = [[x] for x in data_clinc['cls'].unique()]

copus_gomoe = [tokenizer.encode(str(x)).tokens + [e] for x , e in data_gome_values]
copus_go = {idx : copus_gomoe[idx][-1] for idx in range(len(copus_gomoe))}
cls_gomoe = [[x]for x in data_gome["cls"].unique()]


np.random.seed(31)
default_vector = np.random.uniform(-1, 1, 64)
# load_file trước
with open('/content/drive/MyDrive/data-raw/processing/BM250_retrival_Gomoe.pkl' , 'rb' ) as f:
  retrival_gomoe = pickle.load(f)
with open('/content/drive/MyDrive/data-raw/processing/BM25O_Retrival_clinc.pkl' , 'rb') as f:
  retrival_clinc = pickle.load(f)
word2vec_clinc = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_clinc.model')
word2vec_gomoe = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_gomoe.model')
with open('/content/drive/MyDrive/data-raw/processing/copus_clinc.json' , 'r') as f:
  copus_clinc = json.load(f)
with open('/content/drive/MyDrive/data-raw/processing/copus_gomoe.json' , 'r') as f:
  copus_gomoe = json.load(f)


#load -> vector
def read_file_retrival_clinc(text):
  clinc_re = retrival_clinc.get_scores(tokenizer.encode(text).tokens)
  top_indices = sorted(range(len(clinc_re)), key=lambda i: clinc_re[i], reverse=True)[: 9]
  dict_cls = {}
  for inx in top_indices:
    g = copus_clinc[str(inx)]
    dict_cls[g] = dict_cls.get(g , 0) + 1
  clas = max(dict_cls, key=dict_cls.get)
  # print('Max class clinc', clas)
  return word2vec_clinc.wv[clas]


# load gomoe -> vector
def read_file_retrival_gomoe(text):
  gomoe_re = retrival_gomoe.get_scores(tokenizer.encode(text).tokens)
  top_indices = sorted(range(len(gomoe_re)), key=lambda i: gomoe_re[i], reverse=True)[: 9]
  dict_cls = {}
  for inx in top_indices:
    g = copus_gomoe[str(inx)]
    dict_cls[g] = dict_cls.get(g , 0) + 1

  clas = max(dict_cls, key=dict_cls.get)
  # print('Max class gomoe', clas)
  return word2vec_gomoe.wv[clas]

# truncate_padding
pad = tokenizer.token_to_id("[PAD]")
def trun_pad_in(sequen):
  # print('trun_pad_in' , sequen)
  sequen = tokenizer.encode(str(sequen))
  text_ids = sequen.ids[:max_token-1]
  leng = max_token - len(text_ids)
  text_ids += [pad] * leng
  mask = np.where(np.array(text_ids) == 1 , 0 , 1)

  return text_ids, mask

sos = tokenizer.token_to_id("[SOS]")
end = tokenizer.token_to_id("[END]")

def trun_pad_out(sequence):
  # print('trun_pad_out' , sequence)
  tgt = [sos] + tokenizer.encode(str(sequence)).ids + [end]
  tgt_out = tgt[1:]
  tgt_out = tgt_out[: max_token-1]
  tgt_out = tgt_out + [pad] * (max_token - len(tgt_out))

  tgt_in = tgt[:-1]
  tgt_in = tgt_in[: max_token-1]
  tgt_in = tgt_in + [pad] * (max_token - len(tgt_in))
  mask_tgt_in = np.where(np.array(tgt_in) == 1 , 0 , 1)
  return tgt_out , tgt_in , mask_tgt_in


#class loder
class DatasetGo(Dataset):
  def __init__( self, data_in , data_ids_output):
    self.data_in = data_in
    self.data_out = data_ids_output
  def __len__(self):
    return len(self.data_in)

  def __getitem__(self, idx):
    text = self.data_in[idx]
    text_out = self.data_out[idx]

    text_ids , text_mask = trun_pad_in(text)
    text_re_gome = read_file_retrival_gomoe(text)
    text_re_clinc = read_file_retrival_clinc(text)
    tgt_out , tgt_in , mask_tgt_in = trun_pad_out(text_out)
    #print(f"Sample {idx}: text_ids={text_ids}, tgt_out={tgt_out}, tgt_in={tgt_in}")
    #print('text_re_gomoe', text_re_gome.shape)
    return (
        torch.tensor(text_ids , dtype = torch.long),
        torch.tensor(text_re_gome , dtype = torch.float32),
        torch.tensor(text_re_clinc , dtype = torch.float32),
        torch.tensor(text_mask , dtype = torch.long),

        torch.tensor(tgt_out , dtype = torch.long),
        torch.tensor(tgt_in , dtype = torch.long),
        torch.tensor(mask_tgt_in , dtype = torch.float32)
            )


x = data_mul_user['text'].to_list()[8000: 15000]
y = data_mul_bot['text'].to_list()[8000:15000]

Dat = DatasetGo(data_in = x , data_ids_output = y)

dataloder = DataLoader(Dat ,
                       batch_size = 64,
                       shuffle = True ,
                       num_workers= 2,
                       prefetch_factor=2)

# Model chung

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import math
# from kan import *

# possition xác định vị trí các từ
class Possiton(nn.Module):
  def __init__(self, embedding_dim, dropout = 0.3 , max_len = 5000):
    super(Possiton, self).__init__()
    self.dropout = nn.Dropout(p = dropout)
    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
    pe = torch.zeros(max_len, 1, embedding_dim)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)
  def forward(self, x):
    x = x + self.pe[:x.size(1)].squeeze(1)
    return self.dropout(x)

class ModelMain(nn.Module):
  def __init__(
      self,
      embedding_dims,
      embeding_dims_retrival,
      num_heads,
      d_diff ,
      num_layers,
      dropout,
      vocabulary
      ):

    super(ModelMain, self).__init__()

    self.vocabulary = vocabulary
    self.embedding_dims = embedding_dims
    self.norm_emb = nn.RMSNorm(embedding_dims)
    self.norm_ret = nn.RMSNorm(embeding_dims_retrival)

    self.linear_encode = nn.LazyLinear(embedding_dims)

    self.linear_encode_in = nn.Linear(embedding_dims , embedding_dims*2 )
    self.linear_encode_in1 = nn.Linear(embedding_dims*2 , embedding_dims)



# model encoder - layer *L:

    self.model_encode = nn.TransformerEncoder( encoder_layer= nn.TransformerEncoderLayer( d_model = embedding_dims,
                                                                                          nhead = num_heads,
                                                                                          dim_feedforward= d_diff,
                                                                                          dropout = dropout,
                                                                                          activation = F.gelu ), num_layers = num_layers, norm = nn.RMSNorm(embedding_dims))

# model experts 1:

    self.norm_emb1 = nn.RMSNorm(embedding_dims)
    self.norm_model1 = nn.RMSNorm(embedding_dims)
    self.linear_encode_1 = nn.Linear(embedding_dims*2 , embedding_dims)
    self.mode_trans_1 =  nn.TransformerEncoderLayer(d_model = embedding_dims, nhead= 2, dim_feedforward= embedding_dims*2, dropout= 0.3 , activation= F.gelu )

# model 2:

    self.linaer_text_cat2 = nn.LazyLinear(embedding_dims)
    self.norm_emb2 = nn.RMSNorm(embedding_dims)
    self.norm_model2 = nn.RMSNorm(embedding_dims)

    self.linear_2 = nn.Linear(embedding_dims*2 , embedding_dims)

    self.model_trans_2 = nn.TransformerEncoderLayer( d_model= embedding_dims, nhead =  2 , dim_feedforward= embedding_dims *2, dropout = 0.3, activation= F.gelu)


# decode 4
    self.norm_emb3 = nn.RMSNorm(embedding_dims)
    self.norm_model3 = nn.RMSNorm(embedding_dims)

    self.model_deocoder = nn.TransformerDecoder(
        decoder_layer =  nn.TransformerDecoderLayer(
                          d_model = embedding_dims,
                          nhead = num_heads,
                          dim_feedforward= d_diff,
                          dropout = dropout,
                          activation = F.gelu),
        num_layers= num_layers , norm = nn.RMSNorm(embedding_dims)
    )
# Autoregressive9 256 , embedding_dims*2)
    self.linear_autoregre1 = nn.Linear(embedding_dims ,embedding_dims + 256)
    self.linear_autoregre2 = nn.Linear(embedding_dims + 256 , embedding_dims*2)
    self.linear_autoregre = nn.Linear(embedding_dims*2 , vocabulary)
# kan
    # self.kan = KAN(
    #     width = [embedding_dims , embedding_dims*2 , embedding_dims]
    # )
  def embedding_pos(self, x):
    """
    Embedding + Position -> ( seq_length , batch_size , embedding_dims) ( N, S, E)
    """
    embedding = nn.Embedding(self.vocabulary, self.embedding_dims)
    position = Possiton(self.embedding_dims)
    src = embedding(x)
    src = position(src)
    src = src.permute(1, 0, 2)
    # print('embedding')
    return src

  def forward(self, x_emb_retrival , x_embedding ,tgt_out, tgt_mask_key = None,  mask_ids = None ):
    """
    x_emb_retrival ( N , S , E)
    x_embedding ( N , S , E )
    """
    # input
    x_norm_retri_encode = self.norm_ret(x_emb_retrival)
    x_norm_emb_encode = self.norm_emb(x_embedding)
    x_concat = torch.cat((x_norm_retri_encode , x_norm_emb_encode) , dim = -1)
    #print("x_concat", x_concat )
    x_encode = self.linear_encode(x_concat)
    x_encode = F.gelu(x_encode)
    x_encode = self.linear_encode_in(x_encode)
    x_encode = F.gelu(x_encode)
    x_encode = self.linear_encode_in1(x_encode)
    #print("shape " , x_encode.shape)
    # xử lý text model chung
    encode_chung = self.model_encode(x_encode , src_key_padding_mask = mask_ids)
    # print('encode_chung' , encode_chung.shape

    #model 1
    emb_1 = self.norm_emb1(x_encode)
    model_norm_1 = self.norm_model1(encode_chung)
    model_concat_1 = torch.cat((emb_1 , model_norm_1) , dim = -1)
    model_1 = self.linear_encode_1(model_concat_1)
    model_1 = self.mode_trans_1(model_1 , src_key_padding_mask = mask_ids)
    #model 2

    text_2 = self.linaer_text_cat2(x_concat)
    emb_2 = self.norm_emb2(text_2)
    model_norm_2 = self.norm_model2(model_1)

    model_concat_2 = torch.cat((emb_2 , model_norm_2) , dim = -1)
    model_2 = self.linear_2(model_concat_2)
    model_2 = self.model_trans_2(model_2 , src_key_padding_mask = mask_ids)
    #model encode
  #  print("trước tgt_len" , model_2.shape , "tgt.size(1)",tgt_out.size(0))
    tgt_len = tgt_out.size(0)
    tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_len)
    model_3 = self.model_deocoder(tgt = tgt_out , memory = model_2 ,tgt_mask = tgt_mask , tgt_is_causal = True, tgt_key_padding_mask =tgt_mask_key)
# linear FFC
    linear_1 = self.linear_autoregre1(model_3)
    linear_1 = F.gelu(linear_1)
    linear_1 = nn.Dropout(0.3)(linear_1)
    linear_2 = self.linear_autoregre2(linear_1)
    linear_2 = F.gelu(linear_2)
    linear_2 = nn.Dropout(0.3)(linear_2)
    Final = self.linear_autoregre(linear_2)
# linaer KAN


    return Final

In [ ]:
def save_checkpoint( model, loss , optimizer):
  file_name = '/content/drive/MyDrive/data-raw/processing/best_model.pth'
  torch.save(
      {
          'Model' : model,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss.item()
      },
      file_name
  )
  print('Saved checkpoint ------------')


def load_checkpoint(model_ecoder, loss ):
    file_path = '/content/drive/MyDrive/data-raw/processing/best_model.pth'

    checkpoint = torch.load(file_path)

# trainning new start

In [ ]:

def trainning(DataLoader, epochs):
  embedding_dims = 300
  num_classes = 26
  vocab_size = int(tokenizer.get_vocab_size())
  num_layer_main = 10
  num_heads = 10
  dropout = 0.3
  model_encoder = ModelMain(
                              embedding_dims = embedding_dims,
                              num_heads = num_heads,
                              num_layers = num_layer_main,
                              dropout = dropout,
                              embeding_dims_retrival = 64,
                              d_diff= 256,
                              vocabulary = vocab_size)

  model_encoder.train()
  padding_token = tokenizer.token_to_id("[PAD]")
  cretion = nn.CrossEntropyLoss(ignore_index=padding_token , label_smoothing=0.1)
  optimizer_main = torch.optim.Adam(model_encoder.parameters() , lr = 0.001)
  scheduler_main = torch.optim.lr_scheduler.StepLR(optimizer_main, step_size=10, gamma=0.1)
  best_model  = float("inf")
  for epoch in range(epochs):
    loser = []


    for i , (text_ids, text_re_gomoe , text_re_clinc, text_mask, tgt_out, tgt_in, mask_tgt_in) in enumerate(DataLoader):
      """
      text_ids: data inphut text_raw
      text_re_gomoe , text_re_clinc : text_gome và clinc
      text_mask: mask text_ids
      tgt_out: target output
      tgt_in: target input
      mask_tgt_in: mask target input
      """
      with torch.no_grad():
        embedding_text_input = model_encoder.embedding_pos(text_ids)
        embedding_text_ouputin = model_encoder.embedding_pos(tgt_in)
        text_mask = text_mask.type(torch.bool)
        mask_tgt_in = mask_tgt_in.type(torch.bool)
        #tgt_mask_key = tgt_mask_key.type(torch.bool)
        # tạo sequense theo embeding

        #print(f'Lan {i} , text_re_gome {text_re_gomoe.shape} embedding_text_input {embedding_text_input.shape} , embedding_text_ouputin {embedding_text_ouputin.shape}')
       # print(f'text_mask {text_mask.shape} mask_tgt_in {mask_tgt_in.shape}')

        sequen_retrival_gome = text_re_gomoe.unsqueeze(1).repeat(1 , max_token , 1).permute(1 ,0 , 2) #( N, S, E )
        sequen_retrival_clinc = text_re_clinc.unsqueeze(1).repeat(1 , max_token , 1).permute(1, 0, 2) #( N, S, E )


        # print('sequen_retrival_gome' , sequen_retrival_gome.shape)
        # print('sequen_retrival_clinc' , sequen_retrival_clinc.shape)
        # print('embedding_text_input' , embedding_text_input.shape)
        # print('embedding_text_ouputin' , embedding_text_ouputin.shape)
        # print(' shape mask_tgt_in' , mask_tgt_in.shape)
        """
        sequen_retrival_gome torch.Size([32, 200, 64])
  sequen_retrival_clinc torch.Size([32, 200, 64])
        embedding_text_input torch.Size([200, 64, 128])
  embedding_text_ouputin torch.Size([200, 64, 128])
  vector_em_retri torch.Size([32, 200, 128])
        """

        vector_em_retri = torch.cat((sequen_retrival_gome , sequen_retrival_clinc) ,dim = -1  )
        #print('tgt_out' , tgt_out)


        #embeding_final = torch.cat((embedding_text_input ,vector_em_retri) , dim = -1)
        #print('embeding_final' , embeding_final.shape)

  # model trainning

        #print(text_mask.shape)

      out = model_encoder(vector_em_retri,
                          embedding_text_input,
                          embedding_text_ouputin,
                          mask_tgt_in,
                          text_mask
                          )

      # MSE crossentropy out -> (S, N , E ) ( N , S)
      #print('return' , out.shape , 'tgt_out', tgt_out.shape)

      #converts_dim_out = tgt_out.permute(1, 0)
      #print('tgt_out', tgt_out.shape)
      out = out.permute(1 , 2, 0)
      loss = cretion( out , tgt_out ) # model(N, E , S) - tgt(N, S)
      optimizer_main.zero_grad()

      loss.backward()
      optimizer_main.step()
      loser.append(loss.item())

      print(f'epoch {epoch}/ {epochs} batch {i}/ {len(DataLoader)} loss {loss.item()} ')

      if loss.item() < best_model:
        best_model = loss.item()
        save_checkpoint(
            loss = loss,
            model = model_encoder,
            optimizer = optimizer_main
        )
        print(f"da luu model voi loss {loss.item()}")

    print('trung bình loss ' , sum(loser) / len(loser))
m = trainning(dataloder,epochs = 2)

# Load model

In [ ]:
tokenizer.get_vocab_size()

30000

In [ ]:
import torch
import torch.nn as nn

# Khởi tạo model và optimizer
embedding_dims = 300
num_classes = 26
vocab_size = int(tokenizer.get_vocab_size())
num_layer_main = 10
num_heads = 10
dropout = 0.3
max_token = 200
model_main = ModelMain(
    embedding_dims=embedding_dims,
    num_heads=num_heads,
    num_layers=num_layer_main,
    dropout=dropout,
    embeding_dims_retrival=64,
    d_diff=256,
    vocabulary=vocab_size
)

optimizer_main = torch.optim.AdamW(model_main.parameters(), lr=3e-4, weight_decay=1e-4)

# Định nghĩa hàm training
def pre_training(Dataloder, epochs, optimizer_main, model_main):
    # Load model
    file_path = '/content/drive/MyDrive/data-raw/processing/best_model.pth'
    model_load = torch.load(file_path, weights_only = False)
    optimizer_main.load_state_dict(model_load['optimizer_state_dict'])
    model_main.load_state_dict(model_load['model_state_dict'])

    padding_token = tokenizer.token_to_id("[PAD]")
    cretion = nn.CrossEntropyLoss(ignore_index=padding_token, label_smoothing=0.05) # giảm label_smothing để model tập trung hơn vào việc tối ưu
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer_main, step_size=1, gamma=0.95)

    model_main.train()
    best_model = model_load['loss']
 # Đặt giá trị lớn để so sánh loss tốt hơn
    print(' load loss again ' , model_load['loss'] )
    for epoch in range(epochs):
        loser = []
        for i, (text_ids, text_re_gomoe, text_re_clinc, text_mask, tgt_out, tgt_in, mask_tgt_in) in enumerate(Dataloder):

            with torch.no_grad():
                embedding_text_input = model_main.embedding_pos(text_ids)
                embedding_text_ouputin = model_main.embedding_pos(tgt_in)
                text_mask = text_mask.type(torch.bool)
                mask_tgt_in = mask_tgt_in.type(torch.bool)

                # Kiểm tra max_token có tồn tại không

                sequen_retrival_gome = text_re_gomoe.unsqueeze(1).repeat(1, max_token, 1).permute(1, 0, 2)
                sequen_retrival_clinc = text_re_clinc.unsqueeze(1).repeat(1, max_token, 1).permute(1, 0, 2)
                vector_em_retri = torch.cat((sequen_retrival_gome, sequen_retrival_clinc), dim=-1)


            # Forward qua model
            out = model_main(vector_em_retri, embedding_text_input, embedding_text_ouputin, mask_tgt_in, text_mask)


            out = out.permute(1, 2, 0)

            loss = cretion(out, tgt_out)

            optimizer_main.zero_grad()
            loss.backward()
            optimizer_main.step()
            scheduler.step()
            loser.append(loss.item())

            print(f'epoch {epoch}/{epochs} batch {i}/{len(Dataloder)} loss {loss.item()}')

            # Lưu model nếu loss tốt hơn
            if loss.item() < best_model:
                best_model = loss.item()
                save_checkpoint(
                    loss=loss,
                    model=model_main,
                    optimizer=optimizer_main
                )
                print(f"Đã lưu model với loss {loss.item()}")

        print('Trung bình loss:', sum(loser) / len(loser))


"""

text_mask torch.Size([32, 200])
embedding_text_ouputin torch.Size([200, 32, 300])
mask_tgt_in torch.Size([32, 200])
embedding_text_input torch.Size([200, 32, 300])
vector_em_retri torch.Size([200, 32, 64])
sequen_retrival_gome torch.Size([200, 32, 32])
sequen_retrival_clinc torch.Size([200, 32, 32])
trước tgt_len torch.Size([200, 32, 300]) tgt.size(1)

out torch.Size([200, 32, 30000])
out.permute torch.Size([32, 30000, 200])
"""

pre_training(dataloder , 2, optimizer_main, model_main )

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


 load loss again  5.772792816162109
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


epoch 0/2 batch 0/110 loss 5.708310604095459
Saved checkpoint ------------
Đã lưu model với loss 5.708310604095459
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 1/110 loss 5.710229873657227
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 2/110 loss 5.738039493560791
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 3/110 loss 5.572866439819336
Saved checkpoint ------------
Đã lưu model với loss 5.572866439819336
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 4/110 loss 5.750107765197754
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 5/110 loss 5.5857391357421875
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 6/110 loss 5.674737930297852
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 7/110 loss 5.5983991622924805
trước tgt_len torch.Size([200, 64, 300]) tgt.size(1) 200
epoch 0/2 batch 8/110 loss 5.5730204582214

# Inference


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import math
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import pandas as pd
import numpy as np
import pickle
from gensim.models import Word2Vec
import json
max_token = 200
embedding_dims = 300

tokenizer = Tokenizer.from_file('/content/drive/MyDrive/data-raw/processing/file_vocab.json')
with open('/content/drive/MyDrive/data-raw/processing/BM250_retrival_Gomoe.pkl' , 'rb' ) as f:
  retrival_gomoe = pickle.load(f)
with open('/content/drive/MyDrive/data-raw/processing/BM25O_Retrival_clinc.pkl' , 'rb') as f:
  retrival_clinc = pickle.load(f)
word2vec_clinc = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_clinc.model')
word2vec_gomoe = Word2Vec.load('/content/drive/MyDrive/data-raw/processing/Word2vec_cls_gomoe.model')
with open('/content/drive/MyDrive/data-raw/processing/copus_clinc.json' , 'r') as f:
  copus_clinc = json.load(f)
with open('/content/drive/MyDrive/data-raw/processing/copus_gomoe.json' , 'r') as f:
  copus_gomoe = json.load(f)
#load model


def read_file_retrival_clinc(text):
  clinc_re = retrival_clinc.get_scores(tokenizer.encode(text).tokens)
  top_indices = sorted(range(len(clinc_re)), key=lambda i: clinc_re[i], reverse=True)[: 9]
  dict_cls = {}
  for inx in top_indices:
    g = copus_clinc[str(inx)]
    dict_cls[g] = dict_cls.get(g , 0) + 1
  clas = max(dict_cls, key=dict_cls.get)
  # print('Max class clinc', clas)
  return word2vec_clinc.wv[clas]

# load gomoe -> vector
def read_file_retrival_gomoe(text):
  gomoe_re = retrival_gomoe.get_scores(tokenizer.encode(text).tokens)
  top_indices = sorted(range(len(gomoe_re)), key=lambda i: gomoe_re[i], reverse=True)[: 9]
  dict_cls = {}
  for inx in top_indices:
    g = copus_gomoe[str(inx)]
    dict_cls[g] = dict_cls.get(g , 0) + 1
  clas = max(dict_cls, key=dict_cls.get)
  # print('Max class gomoe', clas)
  return word2vec_gomoe.wv[clas]



pad = tokenizer.token_to_id("[PAD]")
def trun_pad_in(sequen):
  # print('trun_pad_in' , sequen)
  sequen = tokenizer.encode(str(sequen))
  text_ids = sequen.ids[:max_token-1]
  leng = max_token - len(text_ids)
  text_ids += [pad] * leng
  mask = np.where(np.array(text_ids) == 1 , 0 , 1)

  return torch.tensor(text_ids, dtype = torch.long), torch.tensor(mask , dtype = torch.long)



checkpoint = torch.load('/content/drive/MyDrive/data-raw/processing/best_model.pth', weights_only=False)
model = checkpoint['Model']
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

def autoregressive_predict(
    model,
    text,
    max_token = 200,
    embedding_dims = embedding_dims, max_len = 200):

  model.eval()

  token_ids , mask_in = trun_pad_in(text)
  mask_in = mask_in.unsqueeze(0).type(torch.bool)
  token_ids = token_ids.unsqueeze(1)
  x_retrival_gome = torch.tensor(read_file_retrival_clinc(text)).unsqueeze(0).unsqueeze(0).repeat( 1 , max_token , 1 )
  x_retrival_clinc = torch.tensor(read_file_retrival_gomoe(text)).unsqueeze(0).unsqueeze(0).repeat( 1 , max_token , 1 )

  print('Shape x_retrival_gome', x_retrival_gome.shape)
  print('Shape x_retrival_clinc', x_retrival_clinc.shape)

  x_emb_retrival = torch.cat((x_retrival_gome ,x_retrival_clinc), dim = -1)


  x_emb = model.embedding_pos(token_ids)
  print('Shape x_emb_retrival', x_emb_retrival.shape)
  print('Shape x_emb', x_emb.shape)

  print('shape mask' , mask_in.shape)

  sos_token_id = tokenizer.token_to_id("[SOS]")
  eos_token_id = tokenizer.token_to_id("[END]")
  tgt_out = torch.tensor([[eos_token_id]])
  tgt_out = model.embedding_pos(tgt_out)
  print('tgt_out' , tgt_out.shape)

  x_emb = x_emb.permute(1, 0, 2)
  tgt_out = tgt_out.permute(1, 0, 2)
  x_emb_retrival = x_emb_retrival.permute(1, 0, 2)
  with torch.no_grad():
    for _ in range(max_len):
      output = model(
                      x_emb_retrival = x_emb_retrival,
                      x_embedding = x_emb,
                      tgt_out = tgt_out,
                      mask_ids = mask_in
                ) # -> (S, N , E )
      print('out_put' , output.shape)
      return output

      next_token_logits = output[-1, 0, :]
      next_token_id = next_token_logits.argmax(dim = -1).item()
      tgt_out = torch.cat([tgt_out, torch.tensor([[next_token_id]])], dim=1)
      if next_token_id == eos_token_id:
        break
    result_ids = tgt_out.squeeze(0).tolist()[1:]
    result_text = tokenizer.decode(result_ids)
    return result_text

input_text = "nhập câu test của bạn ở đây"
linh = autoregressive_predict(model, input_text)

Shape x_retrival_gome torch.Size([1, 200, 32])
Shape x_retrival_clinc torch.Size([1, 200, 32])
Shape x_emb_retrival torch.Size([1, 200, 64])
Shape x_emb torch.Size([1, 200, 300])
shape mask torch.Size([1, 200])
tgt_out torch.Size([1, 1, 300])
out_put torch.Size([1, 1, 30000])
